In [1]:
from google.colab import drive 
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
!pip install transformers==4.28.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 75.9 MB/s eta 0:00:00


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 16.9 MB/s eta 0:00:00


In [ ]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 58.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13707 sha256=723f9e806b69dbfa8681d4cc5b23748fd8d49044bd1ee8d730ba03f13ace5af2
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained('/content/gdrive/MyDrive/Colab Notebooks/Thesis/Models/News/BERT/tokenizer/', use_fast=True, padding=True, truncation=True)
model = AutoModelForMaskedLM.from_pretrained('/content/gdrive/MyDrive/Colab Notebooks/Thesis/Models/News/BERT/', output_hidden_states=True)
model.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(120138, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [ ]:
import torch 

def get_bert_embeddings(input, bert_model):
    with torch.no_grad():
        outputs = bert_model(torch.tensor(input['input_ids']).unsqueeze(dim=0).to(device),
                              token_type_ids=torch.tensor(input['token_type_ids']).unsqueeze(dim=0).to(device),
                              attention_mask=torch.tensor(input['attention_mask']).unsqueeze(dim=0).to(device))
        embeddings = outputs.hidden_states[-1][0, :, :].cpu().numpy()
    return embeddings

In [2]:
import pandas as pd

df_words = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Thesis/Data/Classification/Common/classification_words.csv')
df_words.head()

,WORD,BASE_YEAR,GROUND_TRUTH,NEXT_YEAR
0,железный,2006,0,2007
1,подземный,2009,1,2010
2,катарский,2003,1,2004
3,гуманитарный,2005,0,2006
4,капитальный,2012,0,2013


In [ ]:
import pymorphy2
from datasets import load_from_disk
from tqdm import tqdm
import json
import numpy as np

model.eval()
years = [n for n in range(2000, 2015)]

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

morph = pymorphy2.MorphAnalyzer()

for year in tqdm(years):
    dataset = load_from_disk(f'/content/gdrive/MyDrive/Colab Notebooks/Thesis/Data/News/BERT/dataset_sber_128_{year}')
    words_dict = {'base': {'words': {}, 'count': {}}, 'next': {'words': {}, 'count': {}}}
    part = 0

    words_base = df_words[df_words.BASE_YEAR == year].WORD
    words_next = df_words[df_words.NEXT_YEAR == year].WORD

    for i in tqdm(range(len(dataset['train']))):
        tokenized_text = tokenizer.convert_ids_to_tokens(dataset['train'][i]['input_ids'])
        lemmas = [morph.parse(w)[0].normal_form for w in tokenized_text]
        useful = False

        for word in words_base:
            if word in lemmas:
                useful = True
                break
        if not useful:
            for word in words_next:
                if word in lemmas:
                    useful = True
                    break

        if useful:
            embeds = get_bert_embeddings(dataset['train'][i], model)
            for word in words_base:
                if word in lemmas:
                    for j, lem in enumerate(lemmas):
                        if lem == word:
                            embedding = embeds[j]
                            if word not in words_dict['base']['words']:
                                words_dict['base']['words'][word] = embedding
                                words_dict['base']['count'][word] = 1
                            else:
                                words_dict['base']['words'][word] = words_dict['base']['words'][word] + embedding
                                words_dict['base']['count'][word] += 1
            
            for word in words_next:
                if word in lemmas:
                    for j, lem in enumerate(lemmas):
                        if lem == word:
                            embedding = embeds[j]
                            if word not in words_dict['next']['words']:
                                words_dict['next']['words'][word] = embedding
                                words_dict['next']['count'][word] = 1
                            else:
                                words_dict['next']['words'][word] = words_dict['next']['words'][word] + embedding
                                words_dict['next']['count'][word] += 1

        if i % 10000 == 0 and i != 0:
            for k, v in tqdm(words_dict['base']['words'].items()):
                words_dict['base']['words'][k] = words_dict['base']['words'][k].tolist()
            for k, v in tqdm(words_dict['next']['words'].items()):
                words_dict['next']['words'][k] = words_dict['next']['words'][k].tolist()
            with open(f'/content/gdrive/MyDrive/Colab Notebooks/Thesis/Data/Classification/BERT/word_prototypes/bert_words_dict_news_lem_{part}_{year}.json', 'w', encoding='utf-8') as f:
                json.dump(words_dict, f)
            part += 1
            words_dict = {'base': {'words': {}, 'count': {}}, 'next': {'words': {}, 'count': {}}}        
    
    for i in tqdm(range(len(dataset['validation']))):
        tokenized_text = tokenizer.convert_ids_to_tokens(dataset['validation'][i]['input_ids'])
        lemmas = [morph.parse(w)[0].normal_form for w in tokenized_text]
        useful = False

        for word in words_base:
            if word in lemmas:
                useful = True
                break
        if not useful:
            for word in words_next:
                if word in lemmas:
                    useful = True
                    break
        
        if useful:
            embeds = get_bert_embeddings(dataset['validation'][i], model)
            for word in words_base:
                if word in lemmas:
                    for j, lem in enumerate(lemmas):
                        if lem == word:
                            embedding = embeds[j]
                            if word not in words_dict['base']['words']:
                                words_dict['base']['words'][word] = embedding
                                words_dict['base']['count'][word] = 1
                            else:
                                words_dict['base']['words'][word] = words_dict['base']['words'][word] + embedding
                                words_dict['base']['count'][word] += 1
            for word in words_next:
                if word in lemmas:
                    for j, lem in enumerate(lemmas):
                        if lem == word:
                            embedding = embeds[j]
                            if word not in words_dict['next']['words']:
                                words_dict['next']['words'][word] = embedding
                                words_dict['next']['count'][word] = 1
                            else:
                                words_dict['next']['words'][word] = words_dict['next']['words'][word] + embedding
                                words_dict['next']['count'][word] += 1

    for k, v in tqdm(words_dict['base']['words'].items()):
          words_dict['base']['words'][k] = words_dict['base']['words'][k].tolist()
    for k, v in tqdm(words_dict['next']['words'].items()):
          words_dict['next']['words'][k] = words_dict['next']['words'][k].tolist()
    
    with open(f'/content/gdrive/MyDrive/Colab Notebooks/Thesis/Data/Classification/BERT/word_prototypes/bert_words_dict_news_lem_{part}_{year}.json', 'w', encoding='utf-8') as f:
        json.dump(words_dict, f)

Выходные данные были обрезаны до нескольких последних строк (5000).
 78%|███████▊  | 60000/76998 [1:25:54<13:09, 21.53it/s]

0it [00:00, ?it/s]


100%|██████████| 12/12 [00:00<00:00, 16867.17it/s]

 90%|█████████ | 69652/76998 [1:40:17<07:16, 16.84it/s]

In [3]:
import json
import numpy as np
from tqdm import tqdm

years = [n for n in range(2000, 2015)]

words_embeds = {}

ranges = [3, 4, 4, 4, 5, 7, 7, 7, 9, 11, 10, 10, 11, 10, 8]

for j, year in enumerate(years):
    words_dict = {}

    for i in tqdm(range(ranges[j])):
        if not words_dict:
            with open(f'/content/gdrive/MyDrive/Colab Notebooks/Thesis/Data/Classification/BERT/word_prototypes/bert_words_dict_news_lem_{i}_{year}.json', 'r', encoding='utf-8') as f:
                words_dict = json.load(f)
        else:
            with open(f'/content/gdrive/MyDrive/Colab Notebooks/Thesis/Data/Classification/BERT/word_prototypes/bert_words_dict_news_lem_{i}_{year}.json', 'r', encoding='utf-8') as f:
                words_dict_temp = json.load(f)
            for w in words_dict_temp['base']['words'].keys():
                if w in words_dict['base']['words']:
                    words_dict['base']['words'][w] = np.array(words_dict['base']['words'][w]) + np.array(words_dict_temp['base']['words'][w])
                    words_dict['base']['count'][w] = int(words_dict['base']['count'][w]) + int(words_dict_temp['base']['count'][w]) 
                else:
                    words_dict['base']['words'][w] = np.array(words_dict_temp['base']['words'][w])
                    words_dict['base']['count'][w] = int(words_dict_temp['base']['count'][w])
            for w in words_dict_temp['next']['words'].keys():
                if w in words_dict['next']['words']:
                    words_dict['next']['words'][w] = np.array(words_dict['next']['words'][w]) + np.array(words_dict_temp['next']['words'][w])
                    words_dict['next']['count'][w] = int(words_dict['next']['count'][w]) + int(words_dict_temp['next']['count'][w]) 
                else:
                    words_dict['next']['words'][w] = np.array(words_dict_temp['next']['words'][w])
                    words_dict['next']['count'][w] = int(words_dict_temp['next']['count'][w])
    
    embeds = {'base': {}, 'next': {}}
    for w in words_dict['base']['words'].keys():
        embeds['base'][w] = np.array(words_dict['base']['words'][w]) / int(words_dict['base']['count'][w])
    for w in words_dict['next']['words'].keys():
        embeds['next'][w] = np.array(words_dict['next']['words'][w]) / int(words_dict['next']['count'][w])
    
    words_embeds[f'words_embeds_{year}'] = embeds

100%|██████████| 8/8 [00:01<00:00,  4.35it/s]


In [4]:
df_words['cos_similarity_bert'] = [0 for i in range(df_words.shape[0])]

In [5]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

for i in range(df_words.shape[0]):
    word = df_words.WORD.iloc[i]
    try:
        year_0 = df_words.BASE_YEAR.iloc[i]
        year_1 = year_0 + 1
        embed_1 = words_embeds[f'words_embeds_{year_0}']['base'][word]
        embed_2 = words_embeds[f'words_embeds_{year_1}']['next'][word]
        df_words.loc[i, ('cos_similarity_bert')] = cosine_similarity([embed_1], [embed_2])[0][0]
    except:
        print(word)

катарский
норильский
саяно-шушенский
неназванный
приднестровский
луганский
осетинский
киргизский
рязанский
приемный
принятый
рейтинговый
приемный
басманный
албанский
молодежный
приемный
панкисский
норильский
басманный
панкисский
корейский
садовый
приемный
съемочный
осетинский
вашингтонский
норильский
македонский
комсомольский
сирийский
мертвый
курганский
санкт-петербургский
приемный
норильский
томский
кипрский
октябрьский
дальневосточный
приемный
ливийский
норильский
басманный
балетный
самопровозглашенный
триумфальный
берлинский
невский
амурский
подавляющий
невский
принятый
узбекский
южнокорейский
подавляющий
норильский
тюменский
красноярский
саяно-шушенский
балетный
офшорный
заказной
македонский
иркутский
южноуральский
пушкинский
австралийский
надежный
уставной
летный
узбекский
новгородский


In [ ]:
# Some words from the classification task are composed of several tokens of the BERT models
# Let us find all the occurences for them
df_words_2 = df_words[df_words['cos_similarity_bert'] == 0]

In [ ]:
from datasets import load_from_disk
from ast import literal_eval
from tqdm import tqdm
import json
import numpy as np

model.eval()
years = [n for n in range(2000, 2015)]

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

for year in tqdm(years):
    df = pd.read_csv(f'/content/gdrive/MyDrive/Colab Notebooks/Thesis/Data/News/Common/data_{year}.csv', compression='zip',
                              converters={'tokenized_text': literal_eval, 'lemmas': literal_eval})
    
    words_dict = {'base': {'words': {}, 'count': {}}, 'next': {'words': {}, 'count': {}}}

    words_base = df_words_2[df_words_2.BASE_YEAR == year].WORD.values
    words_next = df_words_2[df_words_2.NEXT_YEAR == year].WORD.values

    for i in tqdm(range(df.shape[0])):
        words_to_check = []
        useful = False
        for word in words_base:
            if word in df.lemmas.iloc[i]:
                useful = True
                words_to_check.append((word, 'base'))
        for word in words_next:
            if word in df.lemmas.iloc[i]:
                useful = True
                words_to_check.append((word, 'next'))

        if useful:
            tokens = tokenizer(' '.join(df.lemmas.iloc[i]), truncation=True, padding=True, max_length=512)
            embeds = get_bert_embeddings(tokens, model)

            for word, indicator in words_to_check:
                t = tokenizer.encode(word)[1:-1]
                for j, tok in enumerate(tokens['input_ids']):
                    if tok == t[0]:
                        embed = embeds[j]
                        for k in range(1, len(t)):
                            if t[k] == tokens['input_ids'][j+k]:
                                embed += embeds[j+k]
                            else:
                                embed = None
                                break 

                        if embed is not None:
                            if word not in words_dict[indicator]['words']:
                                words_dict[indicator]['words'][word] = embed
                                words_dict[indicator]['count'][word] = 1

                            else:
                                words_dict[indicator]['words'][word] = words_dict[indicator]['words'][word] + embed
                                words_dict[indicator]['count'][word] += 1      
    
    for k, v in tqdm(words_dict['base']['words'].items()):
          words_dict['base']['words'][k] = words_dict['base']['words'][k].tolist()
    for k, v in tqdm(words_dict['next']['words'].items()):
          words_dict['next']['words'][k] = words_dict['next']['words'][k].tolist()
    
    with open(f'/content/gdrive/MyDrive/Colab Notebooks/Thesis/Data/Classification/BERT/word_prototypes/bert_words_dict_news_complex_lem_{year}.json', 'w', encoding='utf-8') as f:
        json.dump(words_dict, f)

Выходные данные были обрезаны до нескольких последних строк (5000).
100%|██████████| 51343/51343 [30:57<00:00, 27.65it/s]

100%|██████████| 3/3 [00:00<00:00, 12110.60it/s]

100%|██████████| 44587/44587 [13:27<00:00, 55.23it/s]

100%|██████████| 2/2 [00:00<00:00, 9664.29it/s]

100%|██████████| 44822/44822 [36:32<00:00, 20.44it/s]

100%|██████████| 3/3 [00:00<00:00, 10791.52it/s]

100%|██████████| 46047/46047 [21:24<00:00, 35.85it/s]

100%|██████████| 3/3 [00:00<00:00, 13781.94it/s]

100%|██████████| 43297/43297 [18:51<00:00, 38.27it/s] 

100%|██████████| 4/4 [00:00<00:00, 13584.79it/s]

100%|██████████| 38075/38075 [44:31<00:00, 14.25it/s]

0it [00:00, ?it/s]

100%|██████████| 15/15 [5:54:48<00:00, 1419.25s/it]


In [6]:
years = [n for n in range(2000, 2015)]

words_embeds = {}

for j, year in enumerate(years):
    words_dict = {}

    with open(f'/content/gdrive/MyDrive/Colab Notebooks/Thesis/Data/Classification/BERT/bert_words_dict_news_complex_lem_{year}.json', 'r', encoding='utf-8') as f:
        words_dict = json.load(f)

    embeds = {'base': {}, 'next': {}}
    for w in words_dict['base']['words'].keys():
        embeds['base'][w] = np.array(words_dict['base']['words'][w]) / int(words_dict['base']['count'][w])
    for w in words_dict['next']['words'].keys():
        embeds['next'][w] = np.array(words_dict['next']['words'][w]) / int(words_dict['next']['count'][w])
    
    words_embeds[f'words_embeds_{year}'] = embeds

In [7]:
from sklearn.metrics.pairwise import cosine_similarity

for i in range(df_words.shape[0]):
    word = df_words.WORD.iloc[i]
    if df_words.cos_similarity_bert.iloc[i] == 0:
        try:
            year_0 = df_words.BASE_YEAR.iloc[i]
            year_1 = year_0 + 1
            embed_1 = words_embeds[f'words_embeds_{year_0}']['base'][word]
            embed_2 = words_embeds[f'words_embeds_{year_1}']['next'][word]
            df_words.loc[i, ('cos_similarity_bert')] = cosine_similarity([embed_1], [embed_2])[0][0]
        except:
            print(word)

саяно-шушенский
приемный
принятый
приемный
молодежный
приемный
приемный
съемочный
мертвый
санкт-петербургский
приемный
приемный
самопровозглашенный
подавляющий
принятый
подавляющий
саяно-шушенский
южноуральский
надежный
уставной
летный


In [8]:
df_words[df_words.cos_similarity_bert == 0].count()

WORD                   21
BASE_YEAR              21
GROUND_TRUTH           21
NEXT_YEAR              21
cos_similarity_bert    21
dtype: int64

In [9]:
df_words_1 = df_words.drop(df_words[df_words.cos_similarity_bert == 0].index)

In [10]:
df_words_1.GROUND_TRUTH.value_counts()

0    203
1     41
2     15
Name: GROUND_TRUTH, dtype: int64

In [11]:
df_words_1.reset_index(inplace=True)
df_words_1.drop('index', axis=1, inplace=True)

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, balanced_accuracy_score, precision_score, recall_score
import numpy as np
from tqdm import tqdm

rfc_bert = RandomForestClassifier(n_estimators=105, min_samples_split=5, random_state=42, class_weight='balanced', max_depth=5)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
f1_scores = []
balanced_accuracy_scores = []
precision_scores = []
recall_scores = []

for train_index, test_index in tqdm(skf.split(df_words_1.cos_similarity_bert, df_words_1.GROUND_TRUTH)):
    x_train_fold, x_test_fold = df_words_1.cos_similarity_bert.iloc[train_index], df_words_1.cos_similarity_bert.iloc[test_index]
    y_train_fold, y_test_fold = df_words_1.GROUND_TRUTH.iloc[train_index], df_words_1.GROUND_TRUTH.iloc[test_index]
    rfc_bert.fit(np.expand_dims(np.array(x_train_fold), axis=1), y_train_fold)
    pred = rfc_bert.predict(np.expand_dims(np.array(x_test_fold), axis=1))
    f1_scores.append(f1_score(y_test_fold, pred, average='macro'))
    balanced_accuracy_scores.append(balanced_accuracy_score(y_test_fold, pred))
    precision_scores.append(precision_score(y_test_fold, pred, average='macro'))
    recall_scores.append(recall_score(y_test_fold, pred, average='macro'))

print(f'\nMean F1 score: {np.mean(f1_scores):.4f}')
print(f'Mean balanced accuracy score: {np.mean(balanced_accuracy_scores):.4f}')
print(f'Mean precision score: {np.mean(precision_scores):.4f}')
print(f'Mean recall score: {np.mean(recall_scores):.4f}')

5it [00:01,  3.04it/s]


Mean F1 score: 0.4134
Mean balanced accuracy score: 0.4178
Mean precision score: 0.4189
Mean recall score: 0.4178


In [12]:
df_w2v = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Thesis/Data/Classification/Word2vec/df_cos_w2v.csv')
df_dyn = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Thesis/Data/Classification/Dynamic/df_cos_dyn.csv')
df_dyn.drop(df_dyn[df_dyn.WORD == 'южноуральский'].index, inplace=True)
df_dyn.reset_index(inplace=True)
df_dyn.drop('index', axis=1, inplace=True)

In [13]:
df_words_1['cos_similarity_w2v'] = df_w2v['cos_similarity_w2v']
df_words_1['cos_similarity_dyn'] = df_dyn['cos_similarity_dynamic']

In [14]:
#Combined

from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, balanced_accuracy_score, precision_score, recall_score
import numpy as np
from tqdm import tqdm

rfc_bert = RandomForestClassifier(n_estimators=120, random_state=42, min_samples_split=12, class_weight='balanced', max_depth=4)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
f1_scores = []
balanced_accuracy_scores = []
precision_scores = []
recall_scores = []

for train_index, test_index in tqdm(skf.split(df_words_1.loc[:, ('cos_similarity_bert', 'cos_similarity_w2v')], df_words_1.GROUND_TRUTH)):
    x_train_fold, x_test_fold = df_words_1.loc[train_index, ('cos_similarity_bert', 'cos_similarity_w2v')], \
                                df_words_1.loc[test_index, ('cos_similarity_bert', 'cos_similarity_w2v')]
    y_train_fold, y_test_fold = df_words_1.GROUND_TRUTH.iloc[train_index], df_words_1.GROUND_TRUTH.iloc[test_index]
    rfc_bert.fit(x_train_fold, y_train_fold)
    pred = rfc_bert.predict(x_test_fold)
    f1_scores.append(f1_score(y_test_fold, pred, average='macro'))
    balanced_accuracy_scores.append(balanced_accuracy_score(y_test_fold, pred))
    precision_scores.append(precision_score(y_test_fold, pred, average='macro'))
    recall_scores.append(recall_score(y_test_fold, pred, average='macro'))

print(f'\nMean F1 score: {np.mean(f1_scores):.4f}')
print(f'Mean balanced accuracy score: {np.mean(balanced_accuracy_scores):.4f}')
print(f'Mean precision score: {np.mean(precision_scores):.4f}')
print(f'Mean recall score: {np.mean(recall_scores):.4f}')

5it [00:01,  4.25it/s]


Mean F1 score: 0.4459
Mean balanced accuracy score: 0.4742
Mean precision score: 0.4531
Mean recall score: 0.4742
